# Merging monetization data with the channels dataset

This notebook just merges the df_channels_en.tsv.gz with monetization dataset downloaded using the youtube API, and another dataset from Kaggle

In [90]:
import os
import pandas as pd

In [123]:
root = "~/Downloads/"
path1 = os.path.join(root, "df_channels_en.tsv.gz")
path2 = os.path.join(root, "youtube_monetization_api.csv")
path3 = os.path.join(root, "GlobalYouTubeStatistics.csv")
print(path1)

~/Downloads/df_channels_en.tsv.gz


In [124]:
# Initial channel dataset
channels_df = pd.read_csv(path1, sep="\t")
channels_df.head()

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0,2.087
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.087
2,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0,2.087
3,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0,2.087
4,Sports,2007-05-11,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,48400000,43421,11.0,2.087


Monetization indicators informations downloaded using the youtube API.
This dataset contains the following columns:
- has_affiliate : affiliation to third party companies
- has sponsorships : Content sponsored by companies or brand in exchange for using, demonstrating or discussing their products/services in your video
- has_merchandise : Merchant platforms that helps to sell your branded good to your fans

Normally, youtube propose the following columns:
- affiliate columns: 'affiliate link', '(amazon|shopify|etsy)\.com\/','ref=', 'partner link', 'discount code', 'promo code'
- sponsor columns: 'sponsored by', 'thanks to .+ for sponsoring', 'partner with', '#ad', "#sponsored', 'paid promotion', '\[sponsored\]'
- Merchandise columns: 'merch', 'merchandise', 'shop my', 'buy my', 'teespring', 'represent.com', 'store\.(channel|brand)name'

There is a limitation on the number of requests made per day using the API. So to reduce the number of request, we only perform 3 requests (has_affiliate, has_sponsorships, has_merchandise).

In [125]:
# Monetization indicators informations downloaded using the youtube API
monetization_df = pd.read_csv(path2, sep=",").rename(columns={"channel_id": "channel"})
monetization_df = monetization_df[["channel", "has_affiliate", "has_sponsorships", "has_merchandise"]]
print(f"The shape of channels in the dataset is : {monetization_df.shape}")
monetization_df.head()

The shape of channels in the dataset is : (35408, 4)


,channel,has_affiliate,has_sponsorships,has_merchandise
0,UCpEhnqL0y41EpW2TvWAHD7Q,False,False,True
1,UC295-Dw_tDNtZXFeAPAW6Aw,False,False,False
2,UCJ5v_MCY6GNUBTO8-D3XoAg,False,False,False
3,UCIwFjwMjI0y7PDBVEO9-bkQ,False,False,True
4,UCFFbwnve3yF62-tVXkTyHqg,False,False,False


This dataset was downloaded from Kaggle (https://www.kaggle.com/datasets/nelgiriyewithana/global-youtube-statistics-2023?resource=download) and contains some monetization statistics on 1028 channels.
The most relevant indicators for us are:
- lowest_monthly_earnings: Lowest estimated monthly earnings from the channel
- highest_monthly_earnings: Highest estimated monthly earnings from the channel
- lowest_yearly_earnings: Lowest estimated yearly earnings from the channel
- highest_yearly_earnings: Highest estimated yearly earnings from the channel


In [126]:
# Kaggle dataset
kaggle_df = pd.read_csv(path3, sep=",").rename(columns={"Youtuber" : "name_cc", "category" : "category_cc"})
kaggle_df = kaggle_df[["name_cc", "category_cc", "lowest_monthly_earnings", "highest_monthly_earnings", "lowest_yearly_earnings", "highest_yearly_earnings"]]
print(f"The shape of channels in the dataset is : {kaggle_df.shape}")
kaggle_df.head()

The shape of channels in the dataset is : (1028, 6)


,name_cc,category_cc,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings
0,T-Series,Music,564600.0,9000000.00,6800000.00,1.084000e+08
1,YouTube Movies,Film & Animation,0.0,0.05,0.04,5.800000e-01
2,MrBeast,Entertainment,337000.0,5400000.00,4000000.00,6.470000e+07
3,Cocomelon - Nursery Rhymes,Education,493800.0,7900000.00,5900000.00,9.480000e+07
4,SET India,Shows,455900.0,7300000.00,5500000.00,8.750000e+07


Here, we are merging all datasets

In [127]:
# Merge the initial channels dataset with the kaggle dataset
merged = pd.merge(channels_df, kaggle_df, how = 'left', on = ['name_cc', 'category_cc'])

# Merge with the monetization dataset downloaded using the API
merged_monetized = pd.merge(merged, monetization_df, how = 'left', on = ['channel'])
print(f"The shape of the merged dataset is : {merged_monetized.shape}")
merged_monetized.head()

The shape of the merged dataset is : (136470, 15)


,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,has_affiliate,has_sponsorships,has_merchandise
0,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0,2.087,9800.0,156700.0,117600.0,1900000.00,NaN,NaN,NaN
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.087,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0,2.087,NaN,NaN,NaN,NaN,False,False,True
3,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0,2.087,0.0,0.0,0.0,0.05,False,False,False
4,Sports,2007-05-11,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,48400000,43421,11.0,2.087,178700.0,2900000.0,2100000.0,34300000.00,False,False,False


Write to a csv file

In [131]:
path_to_save = os.path.join(root, "df_channel_with_monetization.csv")
merged_monetized.to_csv(path_to_save, index = False)
print("File saved!")

File saved!
